# Standing Model

In [2]:
import sympy as sp
import numpy as np

## 1 Forward Kinematics

In [3]:
# Symbolic variables 
theta_1,theta_2,theta_3 = sp.symbols("theta_1,theta_2,theta_3 ",real=True)

In [4]:
# Generic Transformation Matrix
def T(theta,x,y):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x],
                      [sp.sin(theta),  sp.cos(theta), y],
                      [            0,              0, 1]])

In [5]:
# Initial Configuration2
p01 = sp.Matrix([[1],[0],[1]]);
p12 = sp.Matrix([[1],[0],[1]]);
p23 = sp.Matrix([[1],[0],[1]]);

In [6]:
# Joint Transformations
T01 = T(theta_1,0,0);
T12 = T(theta_2,1,0)
T23 = T(theta_3,1,0)

In [7]:
# Joint 1 Kinematics
FK1 = (T01 @ p01)[:-1,0]
FK1

Matrix([
[cos(theta_1)],
[sin(theta_1)]])

In [8]:
# Joint 2 Kinematics
FK2 = sp.simplify((T01 @ T12 @ p12))[:-1,0]
FK2

Matrix([
[cos(theta_1) + cos(theta_1 + theta_2)],
[sin(theta_1) + sin(theta_1 + theta_2)]])

In [9]:
# Joint 3 Kinematics
FK3 = sp.simplify((T01 @ T12 @ T23 @ p23))[:-1,0]
FK3

Matrix([
[cos(theta_1) + cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3)],
[sin(theta_1) + sin(theta_1 + theta_2) + sin(theta_1 + theta_2 + theta_3)]])

## 2 Jacobian

In [10]:
# Joint 1 Jacobian
J1 = FK1.jacobian([theta_1])
J1

Matrix([
[-sin(theta_1)],
[ cos(theta_1)]])

In [11]:
# Joint 2 Jacobian
J2 = FK2.jacobian([theta_1,theta_2])
J2

Matrix([
[-sin(theta_1) - sin(theta_1 + theta_2), -sin(theta_1 + theta_2)],
[ cos(theta_1) + cos(theta_1 + theta_2),  cos(theta_1 + theta_2)]])

In [12]:
# Joint 3 Jacobian
J3 = sp.simplify(FK3.jacobian([theta_1,theta_2,theta_3]))
J3

Matrix([
[-sin(theta_1) - sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3), -sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3), -sin(theta_1 + theta_2 + theta_3)],
[ cos(theta_1) + cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3),  cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3),  cos(theta_1 + theta_2 + theta_3)]])

## 3 Agregate Jacobian

In [13]:
J = sp.Matrix([[J1[0],0,0],
               [J1[1],0,0],
               [J2[0,0],J2[0,1],0],
               [J2[1,0],J2[1,1],0],
               [J3[0,0],J3[0,1],J3[0,2]],
               [J3[1,0],J3[1,1],J3[1,2]]])

J

Matrix([
[                                                            -sin(theta_1),                                                          0,                                 0],
[                                                             cos(theta_1),                                                          0,                                 0],
[                                   -sin(theta_1) - sin(theta_1 + theta_2),                                    -sin(theta_1 + theta_2),                                 0],
[                                    cos(theta_1) + cos(theta_1 + theta_2),                                     cos(theta_1 + theta_2),                                 0],
[-sin(theta_1) - sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3), -sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3), -sin(theta_1 + theta_2 + theta_3)],
[ cos(theta_1) + cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3),  cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta

## 4 Mass Matrix

In [14]:
m_1,m_2,m_3,g = sp.symbols('m_1,m_2,m_3,g',real=True)
M = J.transpose() @ sp.diag(m_1,m_1,m_2,m_2,m_3,m_3) @ J
M

Matrix([
[                                                                                                                m_1*sin(theta_1)**2 + m_1*cos(theta_1)**2 + m_2*(-sin(theta_1) - sin(theta_1 + theta_2))**2 + m_2*(cos(theta_1) + cos(theta_1 + theta_2))**2 + m_3*(-sin(theta_1) - sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3))**2 + m_3*(cos(theta_1) + cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3))**2, -m_2*(-sin(theta_1) - sin(theta_1 + theta_2))*sin(theta_1 + theta_2) + m_2*(cos(theta_1) + cos(theta_1 + theta_2))*cos(theta_1 + theta_2) + m_3*(-sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3))*(-sin(theta_1) - sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3)) + m_3*(cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3))*(cos(theta_1) + cos(theta_1 + theta_2) + cos(theta_1 + theta_2 + theta_3)), -m_3*(-sin(theta_1) - sin(theta_1 + theta_2) - sin(theta_1 + theta_2 + theta_3))*sin(theta_1 + theta_2 + theta_3) + m_3*(cos(theta_1) + cos(thet

## 5 Standing Configuration Mass Matrix

In [15]:
M_stand = M.subs([[theta_1,sp.pi/2],[theta_2,0],[theta_3,0]])
M_stand

Matrix([
[m_1 + 4*m_2 + 9*m_3, 2*m_2 + 6*m_3, 3*m_3],
[      2*m_2 + 6*m_3,   m_2 + 4*m_3, 2*m_3],
[              3*m_3,         2*m_3,   m_3]])

## 6 Standing Configuration Gravitational Torque

In [16]:
G_stand = (J.transpose() @ sp.Matrix([[0],[-m_1*g],[0],[-m_2*g],[0],[-m_3*g]])).subs([[theta_1,sp.pi/2],[theta_2,0],[theta_3,0]])
G_stand 

Matrix([
[0],
[0],
[0]])

## 7 Standing Torque

In [17]:
tau_stand = G_stand
tau_stand

Matrix([
[0],
[0],
[0]])